In [1]:
!pip install pandas 
!pip install psycopg2-binary
!pip install sqlalchemy


[notice] A new release of pip available: 22.2.2 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
  Using cached psycopg2_binary-2.9.9-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.0 MB)

[notice] A new release of pip available: 22.2.2 -> 24.1.1
[notice] To update, run: pip install --upgrade pip
  Using cached SQLAlchemy-2.0.31-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.1 MB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)
  Using cached greenlet-3.0.3-cp39-cp39-manylinux_2_24_x86_64.manylinux_2_28_x86_64.whl (614 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.3.0
    Uninstalling typing_extensions-4.3.0:
      Successfully uninstalled typing_extensions-4.3.0
ERROR: Could not install packages due to an OSError: [Errno 13] Permission denied: '/usr/local/include/python3.9/greenlet'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip avai

In [3]:
import boto3
from botocore.exceptions import ClientError
import json

def get_secret():

    secret_name = "DBCreds"
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        raise e

    secret = get_secret_value_response['SecretString']
    
    # Parse the secret string to get the credentials
    secret_dict = json.loads(secret)
    username = secret_dict['username']
    password = secret_dict['password']
    host = secret_dict['host']
    port = secret_dict['port']
    dbname = secret_dict['dbname']

    return username, password, host, port, dbname


(user,pswd,host,port,db) = get_secret()

In [ ]:
import json
from kafka import KafkaConsumer
import psycopg2
from psycopg2.extras import execute_values

# PostgreSQL database connection details
db_config = {
    'dbname': db,
    'user': user,
    'password': pswd,
    'host': host,
    'port': port
}

# Kafka topic
postgres_topic = "postgresql"
brokers = [f"{host}:9092"]

# Create a Kafka consumer
consumer = KafkaConsumer(
    postgres_topic,
    bootstrap_servers=brokers,
    value_deserializer=lambda message: json.loads(message.decode('utf-8'))
)

# Connect to PostgreSQL
try:
    conn = psycopg2.connect(**db_config)
    cursor = conn.cursor()
    print("Connected to PostgreSQL successfully.")
except Exception as e:
    print(f"Failed to connect to PostgreSQL: {e}")
    exit()

try:
    print("Starting to consume messages.")
    for message in consumer:
        data = message.value
        data['datatype'] = 2
        print(f"Received message: {data}")
        # Insert data into PostgreSQL
        try:
            columns = data.keys()
            values = [tuple(data.values())]
            insert_query = f"INSERT INTO malware_data ({', '.join(columns)}) VALUES %s"
            execute_values(cursor, insert_query, values)
            conn.commit()
            print("Data inserted successfully")
        except Exception as e:
            print(f"Failed to insert data: {e}")
            conn.rollback()
except KeyboardInterrupt:
    print("Consumer interrupted.")
finally:
    # Close PostgreSQL connection
    if cursor:
        cursor.close()
    if conn:
        conn.close()
        print("PostgreSQL connection closed.")


Connected to PostgreSQL successfully.
Starting to consume messages.
Received message: {'SizeOfOptionalHeader': '224.0', 'MinorLinkerVersion': '0.0', 'SizeOfCode': '361984.0', 'SizeOfInitializedData': '115712.0', 'SizeOfUninitializedData': '0.0', 'BaseOfCode': '4096.0', 'BaseOfData': '372736.0', 'ImageBase': '4194304.0', 'SectionAlignment': '4096.0', 'MajorOperatingSystemVersion': '0.0', 'MinorImageVersion': '0.0', 'MajorSubsystemVersion': '1.0', 'SizeOfHeaders': '1024.0', 'CheckSum': '485887.0', 'Subsystem': '16.0', 'DllCharacteristics': '1024.0', 'SizeOfStackReserve': '1048576.0', 'SizeOfStackCommit': '4096.0', 'SizeOfHeapReserve': '1048576.0', 'NumberOfRvaAndSizes': '16.0', 'SectionsNb': '8.0', 'SectionsMinEntropy': '3.60742957555', 'SectionsMaxEntropy': '7.22105072892', 'SectionMaxRawsize': '325120.0', 'SectionsMinVirtualsize': '896.0', 'SectionMaxVirtualsize': '551848.0', 'ImportsNb': '0.0', 'ImportsNbOrdinal': '0.0', 'ExportNb': '0.0', 'ResourcesNb': '4.0', 'ResourcesMinEntropy': 